# Generación de Data Única referente a Rol

Unificacion de la data de ingresos y terminos por Rol, además de transformación de algunos datos para normalizar nombres en columnas

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata
from src.data import transformdata

In [2]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [3]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/processes_ListadoTribunales.feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

# DATA INGRESO Y TERMINO ROL PENAL

### INGRESOS X ROL

In [4]:
df_ingresos_rol = pd.read_feather(f"{path_interim}/clean_IngresosRol.feather")
df_ingresos_rol.sort_values('FECHA INGRESO')

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,MES INGRESO,AÑO INGRESO
1533510,1533758,304386,50,C.A. DE TEMUCO,938,JUZGADO DE GARANTIA DE PITRUFQUEN,1-2015,Ordinaria,2015-01-01,ene-15,2015
1395150,1395397,166025,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA DE RANCAGUA,16-2015,Ordinaria,2015-01-01,ene-15,2015
1395149,1395396,166024,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA DE RANCAGUA,15-2015,Ordinaria,2015-01-01,ene-15,2015
1395148,1395395,166023,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA DE RANCAGUA,14-2015,Ordinaria,2015-01-01,ene-15,2015
1395147,1395394,166022,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA DE RANCAGUA,13-2015,Ordinaria,2015-01-01,ene-15,2015
...,...,...,...,...,...,...,...,...,...,...,...
828839,829003,222028,40,C.A. DE TALCA,964,JUZGADO DE GARANTIA DE TALCA,10905-2019,Ordinaria,2019-12-31,dic-19,2019
828838,829002,222027,40,C.A. DE TALCA,964,JUZGADO DE GARANTIA DE TALCA,10904-2019,Ordinaria,2019-12-31,dic-19,2019
828837,829001,222026,40,C.A. DE TALCA,964,JUZGADO DE GARANTIA DE TALCA,10903-2019,Ordinaria,2019-12-31,dic-19,2019
828845,829009,222034,40,C.A. DE TALCA,964,JUZGADO DE GARANTIA DE TALCA,10911-2019,Ordinaria,2019-12-31,dic-19,2019


In [5]:
# Normalizo nombres

df_ingresos_rol['TRIBUNAL'] = df_ingresos_rol['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 2980483/2980483 [00:18<00:00, 160318.16it/s]


In [6]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_ingresos_rol['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

In [7]:
filtro_null = df_ingresos_rol[df_ingresos_rol['FECHA INGRESO'].isnull()]
print(f"Existen {len(filtro_null)} causas con Fechas Null")

Existen 0 causas con Fechas Null


### TERMINO X ROL

In [8]:
df_termino_rol = pd.read_feather(f"{path_interim}/clean_TerminosRol.feather")
df_termino_rol.sort_values('FECHA INGRESO')

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
337822,337862,337911,50,C.A. DE TEMUCO,214,JUZGADO DE LETRAS Y GARANTIA DE PUCON,1059-2017,Ordinaria,1900-01-01,2018-10-01,-22164,DECLARA SOBRESEIMIENTO DEFINITIVO.,oct-18,2018,1
299878,299917,299951,46,C.A. DE CONCEPCION,1080,JUZGADO DE GARANTIA DE TALCAHUANO,2398-2017,Ordinaria,1900-01-01,2018-05-31,-22287,DECLARA SOBRESEIMIENTO DEFINITIVO.,may-18,2018,1
401020,401065,401124,56,C.A. DE PUERTO MONTT,662,JUZGADO DE LETRAS Y GARANTIA DE QUELLON,684-2018,Ordinaria,1900-01-01,2018-08-21,-22205,SENTENCIA,ago-18,2018,1
2767837,2768176,91242,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,1-2000,Ordinaria,2000-12-20,2015-06-02,5277,CERTIFICA CUMPLIMIENTO ART. 468.,jun-15,2015,1
2770323,2770662,93728,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,3-2000,Ordinaria,2000-12-21,2015-06-02,5276,NO PERSEVERAR EN EL PROCEDIMIENTO.,jun-15,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094026,1094145,418737,56,C.A. DE PUERTO MONTT,1089,JUZGADO DE GARANTIA DE CASTRO,3123--2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
1025618,1025733,350306,50,C.A. DE TEMUCO,216,JUZGADO DE LETRAS Y GARANTIA DE CARAHUE,1377--2019,Ordinaria,2019-12-31,2019-12-31,0,DECLARA INCOMPETENCIA RPA,dic-19,2019,1
820414,820516,145052,30,C.A. DE VALPARAISO,1047,JUZGADO DE GARANTIA DE QUILPUE,4679--2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
1201934,1202065,526657,90,C.A. DE SANTIAGO,1226,7º JUZGADO DE GARANTIA DE SANTIAGO,22338--2019,Ordinaria,2019-12-31,2019-12-31,0,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,dic-19,2019,1


In [9]:
filtro_null = df_termino_rol[df_termino_rol['FECHA INGRESO'].isnull()]
print(f"Existen {len(filtro_null)} causas con ingresos NULL")

Existen 0 causas con ingresos NULL


In [10]:
df_termino_rol.drop(filtro_null.index, axis=0, inplace=True)

In [11]:
filtro_fecha = df_termino_rol[df_termino_rol['FECHA INGRESO'] <='2014-12-31']
print(f"Existen {len(filtro_fecha)} causas con ingresos anteriores a 01-01-2015")

Existen 410874 causas con ingresos anteriores a 01-01-2015


In [12]:
df_termino_rol.drop(filtro_fecha.index, axis=0, inplace=True)

In [13]:
# Normalizo nombres

df_termino_rol['TRIBUNAL'] = df_termino_rol['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 2945536/2945536 [00:16<00:00, 174378.28it/s]


In [14]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_termino_rol['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

In [15]:
df_termino_rol[df_termino_rol['DURACION CAUSA']<0]

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
639,639,639,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10153-2018,Ordinaria,2018-12-11,2018-11-30,-11,DECLARA INCOMPETENCIA.,nov-18,2018,1
640,640,640,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10154-2018,Ordinaria,2018-12-11,2018-11-30,-11,DECLARA INCOMPETENCIA.,nov-18,2018,1
641,641,641,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10155-2018,Ordinaria,2018-12-11,2018-12-03,-8,DECLARA INCOMPETENCIA.,dic-18,2018,1
642,642,642,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10156-2018,Ordinaria,2018-12-11,2018-12-03,-8,DECLARA INCOMPETENCIA.,dic-18,2018,1
916,916,916,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10413-2018,Ordinaria,2018-12-20,2018-12-05,-15,DECLARA INCOMPETENCIA.,dic-18,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354289,3354710,677837,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6341-2015,Ordinaria,2015-12-15,2015-11-20,-25,DECLARA INCOMPETENCIA.,nov-15,2015,1
3354359,3354780,677907,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6412-2015,Ordinaria,2015-12-18,2015-12-15,-3,DECLARA INCOMPETENCIA RPA,dic-15,2015,1
3354419,3354840,677967,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6497-2015,Ordinaria,2015-12-22,2015-12-17,-5,DECLARA INCOMPETENCIA RPA,dic-15,2015,1
3354479,3354900,678027,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,6534-2015,Ordinaria,2015-12-22,2015-11-25,-27,DECLARA INCOMPETENCIA.,nov-15,2015,1


In [16]:
df_termino_rol = df_termino_rol.progress_apply(transformdata.fechas_cambiadas, axis=1)

100%|██████████| 2945536/2945536 [06:35<00:00, 7442.69it/s] 


In [17]:
df_termino_rol[df_termino_rol['DURACION CAUSA']<0]

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS


## CARGA DATA A ARCHIVOS FUTHER

In [18]:
# Directorio donde se guardaran archivos feather
df_ingresos_rol.reset_index(inplace=True, drop=True)
df_termino_rol.reset_index(inplace=True, drop=True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_ingresos_rol.to_feather(f'{path_processed}/processes_IngresosRol.feather')
df_termino_rol.to_feather(f'{path_processed}/processes_TerminosRol.feather')